In [14]:
import cv2
import numpy as np
import os

# KNN algo code

In [15]:
def distance(v1, v2):
    # Eucledian 
    return np.sqrt(((v1-v2)**2).sum())
def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]
    
    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]

In [16]:
cam=cv2.VideoCapture(0)  #to intialize camera

In [17]:
face_cascade=cv2.CascadeClassifier("/Users/nambi/OneDrive/Desktop/AIandMLcourse/Face recognition/haarcascade_frontalface_alt.xml")

In [ ]:
skip=0
face_data=[]
labels=[]
class_id=0   #labels to given file
names={}    #to map ids and names
          
dataset_path="/Users/nambi/OneDrive/Desktop/AIandMLcourse/Face recognition"


# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #Create a mapping btw class_id and name
        names[class_id] = fx[:-4]      #to get names meams removing ".npy" from file name
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        #Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)


while True:
    ret,frame=cam.read() #ret :: bollean value to check whether frame caputured properly or not
                         #frame :: frame value
    if ret==False:
        continue
    
    Gray_Frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces=face_cascade.detectMultiScale(Gray_Frame,1.3,5)
    if len(faces)==0:
        continue
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        x,y,w,h=face
        
        #now to extract only face area and to store it 
        offset=10
        face_area=Gray_Frame[y-offset:y+offset,x-offset:x+w+offset]
        face_area=cv2.resize(face_area,(100,100))
        
#         predicted label
        out = knn(trainset,face_area.flatten())
        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(Gray_Frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(Gray_Frame,(x,y),(x+w,y+h),(0,255,255),2)
    
    cv2.imshow("Faces",frame)
    
    key_pressed=cv2.waitKey(1)& 0xFF   #to exit while loop i intiated waitkey
    if key_pressed==ord('q'):         #ord function gives ascii value of any key
        break
        

cam.release()
cv2.destroyAllWindows()

